In [1]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


In [5]:
A = mg.read_csv('DBLP_cleaned.csv', key='id')

In [4]:
B = mg.read_csv('ACM_cleaned.csv', key='id')

In [6]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(dblp, acm, 'year', 'year', ['title', 'authors', 'year', 'venue'],
                   ['title', 'authors', 'year', 'venue'])

In [7]:
# sample C
S = mg.sample_table(C, 400)

In [8]:
# label table
#G = mg.label_table(S, 'gold')

In [15]:
# for demo load a pre-labeled set
G = mg.read_csv('acm_dblp_label.csv', key='_id', ltable=A, rtable=B)

In [17]:
#plan

# 1. sample candidate set --> S
# 2. label S --> L
# 3. Split L into development I, and evaluation set J
# 4. Select best matcher Y, using I 
# 5. Add triggers to Y ---> Z
# 6. Evaluate Z using J

In [20]:
IJ = mg.train_test_split(G, train_proportion=0.7)
I = IJ['train']
J = IJ['test']

In [21]:
len(I), len(J)

(278, 120)

In [22]:
# selecting the best learning matcher
# 1. Create a set of ML-matchers
# 2. Generate features --> feature_table
# 3. Extract features using I and feature_table
# 4. Select best matcher using CV
# 5. Debug the selected matcher (and repeat the above steps)

In [29]:
feature_table = mg.get_features_for_matching(A, B)
mg._atypes1['venue']='str_bt_5w_10w'
feature_table = mg.get_features(A, B, mg._atypes1, mg._atypes2, mg._match_c, mg._match_t, 
                               mg._match_s)

In [30]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x115...,from magellan.feature.simfunctions import *\nf...
1,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0...,from magellan.feature.simfunctions import *\nf...
2,title_title_mel,title,title,None,None,monge_elkan,<function title_title_mel at 0x115c31938>,from magellan.feature.simfunctions import *\nf...
3,title_title_lev,title,title,None,None,lev,<function title_title_lev at 0x115c31ed8>,from magellan.feature.simfunctions import *\nf...
4,authors_authors_jac_qgm_3_qgm_3,authors,authors,qgm_3,qgm_3,jaccard,<function authors_authors_jac_qgm_3_qgm_3 at 0...,from magellan.feature.simfunctions import *\nf...
5,authors_authors_cos_dlm_dc0_dlm_dc0,authors,authors,dlm_dc0,dlm_dc0,cosine,<function authors_authors_cos_dlm_dc0_dlm_dc0 ...,from magellan.feature.simfunctions import *\nf...
6,authors_authors_mel,authors,authors,None,None,monge_elkan,<function authors_authors_mel at 0x115c31aa0>,from magellan.feature.simfunctions import *\nf...
7,authors_authors_lev,authors,authors,None,None,lev,<function authors_authors_lev at 0x115c317d0>,from magellan.feature.simfunctions import *\nf...
8,venue_venue_jac_qgm_3_qgm_3,venue,venue,qgm_3,qgm_3,jaccard,<function venue_venue_jac_qgm_3_qgm_3 at 0x115...,from magellan.feature.simfunctions import *\nf...
9,venue_venue_cos_dlm_dc0_dlm_dc0,venue,venue,dlm_dc0,dlm_dc0,cosine,<function venue_venue_cos_dlm_dc0_dlm_dc0 at 0...,from magellan.feature.simfunctions import *\nf...


In [57]:
f = feature_table.ix[[10, 12, 13, 14]]

In [58]:
f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
10,venue_venue_mel,venue,venue,None,None,monge_elkan,<function venue_venue_mel at 0x115663c08>,from magellan.feature.simfunctions import *\nf...
12,year_year_exm,year,year,None,None,exact_match,<function year_year_exm at 0x115663b18>,from magellan.feature.simfunctions import *\nf...
13,year_year_anm,year,year,None,None,abs_norm,<function year_year_anm at 0x115663aa0>,from magellan.feature.simfunctions import *\nf...
14,year_year_lev,year,year,None,None,lev,<function year_year_lev at 0x115663a28>,from magellan.feature.simfunctions import *\nf...


In [81]:
mg.init_jvm()

True

In [59]:
# extract features
H = mg.extract_feature_vecs(G, feature_table=f, attrs_after='gold')

In [60]:
# impute values
H.fillna(0, inplace=True)

In [61]:
# Create ML matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

In [64]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall') 

In [65]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,SVM_06904021795058376441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.804878,0.658537,0.810811,0.694444,0.604651,0.714664
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.685714,0.702703,0.702128,0.717949,0.750000,0.711699
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.729167,0.771429,0.771429,0.648649,0.651163,0.714367
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,0.641026,0.720930,0.815789,0.589744,0.794872,0.712472


In [68]:
# Split feature vectors to train and test
UV = mg.train_test_split(H, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']